## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate       source  \
0  2025-09-24-18-51-06 +0000       nypost   
1  2025-09-24-18-50-43 +0000         wapo   
2  2025-09-24-18-50-43 +0000         wapo   
3  2025-09-24-18-50-07 +0000  startribune   
4  2025-09-24-18-46-33 +0000          nyt   

                                               title  \
0  Family challenges ICE’s presumed cause of deat...   
1  Texas governor: ‘Horrific’ shooting won’t stop...   
2  Live updates: Two killed in Dallas ICE facilit...   
3  Judge scolds Justice Department over public st...   
4  The D.C. Region Has Faced an Outsized Toll Fro...   

                                                link  
0  https://nypost.com/2025/09/24/us-news/family-c...  
1                     https://www.washingtonpost.com  
2  https://www.washingtonpost.com/nation/2025/09/...  
3  https://www.startribune.com/judge-scolds-justi...  
4  https://www.nytimes.com/2025/09/24/us/politics...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
44    trump     64
183  kimmel     16
2       ice     14
303     new     13
419   jimmy     12

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
236  2025-09-24-01-43-00 +0000     wsj   
105  2025-09-24-14-05-20 +0000  nypost   
120  2025-09-24-13-01-18 +0000     cbc   
212  2025-09-24-05-04-00 +0000     wsj   
182  2025-09-24-09-30-00 +0000     wsj   

                                                 title  \
236  When Trump announced changes to the H-1B visa ...   
105  Trump threatens to sue ABC over Jimmy Kimmel’s...   
120  Zelenskyy warns UN of arms race, as Russia sco...   
212  Kimmel Makes Emotional Return to ABC With Trum...   
182  Stephen Miran set out his case for lower inter...   

                                                  link  score  
236  https://www.wsj.com/politics/policy/air-india-...    134  
105  https://nypost.com/2025/09/24/us-news/trump-th...    121  
120  https://www.cbc.ca/news/world/ukraine-war-zele...    119  
212  https://www.wsj.com/business/media/jimmy-kimme...    119  
182  https://www.wsj.com/economy/central-banking/st...    114

## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
236    134  2025-09-24-01-43-00 +0000     wsj   
80      76  2025-09-24-15-46-02 +0000     bbc   
2       65  2025-09-24-18-50-43 +0000    wapo   
211     54  2025-09-24-05-11-13 +0000  nypost   
120     53  2025-09-24-13-01-18 +0000     cbc   
167     45  2025-09-24-10-00-00 +0000  nypost   
91      40  2025-09-24-15-06-31 +0000     bbc   
93      39  2025-09-24-15-00-55 +0000  nypost   
244     31  2025-09-24-00-42-57 +0000     cbc   
88      30  2025-09-24-15-32-02 +0000  nypost   

                                                 title  \
236  When Trump announced changes to the H-1B visa ...   
80   Jimmy Kimmel calls out 'anti-American' threats...   
2    Live updates: Two killed in Dallas ICE facilit...   
211  Typhoon Ragasa batters Hong Kong and southern ...   
120  Zelenskyy warns UN of arms race, as Russia sco...   
167  Vivek Ramaswamy tells ‘Pod Force One’ he was o...   
91   Israeli forces near Gaza City centre as Hamas ...   
93   New home sales surge over 20% to 3-year high a...   
244  Carney says he will meet China's president 'at...   
88   News coverage of Charlie Kirk’s memorial racks...   

                                                  link  
236  https://www.wsj.com/politics/policy/air-india-...  
80   https://www.bbc.com/news/articles/c9dx6dlelw8o...  
2    https://www.washingtonpost.com/nation/2025/09/...  
211  https://nypost.com/2025/09/24/world-news/typho...  
120  https://www.cbc.ca/news/world/ukraine-war-zele...  
167  https://nypost.com/2025/09/24/us-news/vivek-ra...  
91   https://www.bbc.com/news/articles/c4gk9dreldxo...  
93   https://nypost.com/2025/09/24/business/new-hom...  
244  https://www.cbc.ca/news/politics/carney-china-...  
88   https://nypost.com/2025/09/24/business/news-co...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
